In [1]:
# Importar las librerías necesarias
import tensorflow as tf
import numpy as np
from tensorflow import keras
#import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.callbacks import EarlyStopping


from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from Limpiezadatos import df_sin_nulos

c:\Users\jpcan\OneDrive\Documentos\Andes Universidad\Analitica computacional\Proyecto_final_github\Limpiezadatos.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, engine)
c:\Users\jpcan\OneDrive\Documentos\Andes Universidad\Analitica computacional\Proyecto_final_github\Limpiezadatos.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sin_nulos.drop(columns=columnas_a_eliminar, inplace=True)
c:\Users\jpcan\OneDrive\Documentos\Andes Universidad\Analitica computacional\Proyecto_final_github\Limpiezadatos.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [2]:
import pandas as pd
import numpy as np



# Dividir en X e Y
Y = df_sin_nulos['punt_sociales_ciudadanas']
columnas_a_excluir = ['punt_ingles','punt_matematicas', 'punt_sociales_ciudadanas', 'punt_c_naturales', 'punt_lectura_critica','cole_mcpio_ubicacion','cole_nombre_sede','periodo']
#'punt_matematicas', 'punt_sociales_ciudadanas', 'punt_c_naturales', 'punt_lectura_critica'
X = df_sin_nulos.drop(columns=columnas_a_excluir, axis=1)
# Convertir las columnas categóricas en variables dummy
X_dummies = pd.get_dummies(X)
X_dummies = X_dummies.astype(int)


# Convertir a arrays de numpy
X_dummies = np.array(X_dummies)
Y = np.array(Y)



X_dummies = X_dummies[:30000]
Y = Y[:30000]


In [3]:
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_dummies, Y, test_size=0.3, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=0.3, random_state=42)

# Escalar los datos (opcional, pero es común en redes neuronales)
std_scl = StandardScaler()
std_scl.fit(X_train)

print(X_train[0:3,])
X_train = std_scl.transform(X_train)
print(X_train[0:3,])
X_valid = std_scl.transform(X_valid)
X_test = std_scl.transform(X_test)

[[17  0  1 ...  1  0  1]
 [19  0  1 ...  1  1  0]
 [16  0  1 ...  0  0  1]]
[[-0.18332631 -0.08522476  0.08522476 ...  0.34171142 -0.28815359
   0.28815359]
 [ 0.35874284 -0.08522476  0.08522476 ...  0.34171142  3.47037149
  -3.47037149]
 [-0.45436089 -0.08522476  0.08522476 ... -2.92644595 -0.28815359
   0.28815359]]


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError  # Cambio en la función de pérdida
from sklearn.model_selection import train_test_split
import itertools

tf.keras.backend.clear_session()

# Definir los datos de entrenamiento y validación (X_train, X_valid, y_train, y_valid)
# Supongamos que ya están definidos

# Definir la configuración de hiperparámetros a explorar
hidden_layers = [1, 2, 3]  # Cantidad de capas ocultas
neurons_per_layer = [200, 128, 500]  # Número de neuronas por capa

# Inicializar un diccionario para almacenar los resultados
results = {}

# Iterar sobre las combinaciones de hiperparámetros
for num_layers, num_neurons in itertools.product(hidden_layers, neurons_per_layer):
    # Crear modelo
    model = Sequential()
    model.add(Dense(num_neurons, input_shape=(X_train.shape[1],), activation="relu"))
    for _ in range(num_layers - 1):
        model.add(Dense(num_neurons, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(Dense(1, activation='softplus'))

    # Compilar modelo
    model.compile(loss=MeanSquaredError(),  
                  optimizer="adam",
                  metrics=["mean_absolute_error"]) 

    # Entrenar modelo
    history = model.fit(X_train, y_train, epochs=10,
                        validation_data=(X_valid, y_valid),
                        verbose=0)

    # Evaluar modelo en datos de validación
    val_loss, val_mae = model.evaluate(X_valid, y_valid, verbose=0)

    # Almacenar los resultados en el diccionario
    results[(num_layers, num_neurons)] = {'loss': val_loss, 'mean_absolute_error': val_mae, 'model': model}

# Encontrar la mejor configuración
best_configuration = min(results, key=lambda x: results[x]['loss'])
best_model = results[best_configuration]['model']

print("Mejor configuración:")
print("Número de capas ocultas:", best_configuration[0])
print("Número de neuronas por capa:", best_configuration[1])
print("Loss:", results[best_configuration]['loss'])
print("Mean Absolute Error:", results[best_configuration]['mean_absolute_error'])

# Serializar el mejor modelo a un archivo .keras
best_model.save('C:/Users/jpcan/OneDrive/Documentos/Andes Universidad/Analitica computacional/Proyecto_final_entregables/Modelos/Mejor_modelo_sociales.keras')







Mejor configuración:
Número de capas ocultas: 3
Número de neuronas por capa: 200
Loss: 94.68775177001953
Mean Absolute Error: 7.654648303985596


In [ ]:
import matplotlib.pyplot as plt
# Preparar los datos para el gráfico de barras
configurations = list(results.keys())
mae_values = [results[config]['mean_absolute_error'] for config in configurations]
x_ticks = [f'{config[0]} capas, {config[1]} neuronas' for config in configurations]

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
bars = plt.bar(x_ticks, mae_values, color='white', edgecolor='green')

# Resaltar la mejor combinación en verde oscuro
best_index = configurations.index(best_configuration)
bars[best_index].set_color('darkgreen')

# Añadir etiquetas y título
plt.xlabel('Configuración')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('Comparación de configuraciones según MAE')

# Rotar etiquetas del eje x para mejor visualización
plt.xticks(rotation=45, ha='right', fontsize=10)

# Mostrar el gráfico
plt.tight_layout()
plt.show()